# Hidden Markov Modeling
Estimate hidden Markov model (HMM) parameters and train a model on tracked cells. 

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import ermine as em
import math
from matplotlib import pyplot as plt
from pySPT.Analysis import hiddenMarkovModeling
from pySPT.widgets import widgetHMM, saveHMM
import warnings
warnings.filterwarnings("ignore")
widget_init_hmm = widgetHMM.WidgetInitHMM(dt=0.02, init_n_start=1, init_n_end=4, x_axis=300)
# define the colors of states here:
color_palette_hex = ["#4169e1", "#228b22", "#ff8c00", "#8b008b", "#ff4500", "#fb49b0", "#cacaca", "#b24502"]
widget_hmm = widgetHMM.WidgetHMM(dt=0.02, n_states=2, diffs="0.11, 0.8", weights="0.3, 0.7", min_len=4, immob_threshold=25, epsilon=20, x_axis=300, graphviz_bin=r"C:\Program Files (x86)\Graphviz2.38\bin")
seed = 42
np.random.seed(seed)

## Initial parameter estimation
Estimate the number of hidden states and yield initial parameters.

Initial parameters (weights and diffusion coefficients) for the hidden markov model are estimated by fitting the jump distance distribution of cells with a mixture model. The distribution can be fitted with different numbers of states, defined by the min and max number of states. For example, if min = 1 and max = 4, the distribution is fitted with 1, 2, 3, or 4 subpopulations and information criteria are calculated giving first hints about the number of states. The averaged weights and diffusion coefficent of all cells are calculated and can be used as initial parameters for the respective number of hidden states. <br/>

*Directory:* All tracked files will be analyzed within defined input directory. <br/>
*Camera integration time [s]:* The camera integration time of measurements in seconds. <br/>
*Min number of states:* Minimum number of mixed models. <br/>
*Max number of states:* Maximum number of mixed models. <br/>

In [2]:
display(widget_init_hmm.dir_box, widget_init_hmm.dir_button)
widget_init_hmm.dir_button.on_click(widget_init_hmm.open_dir)
widget_init_hmm.dir_box.observe(widget_init_hmm.change_dir_box)
display(widget_init_hmm.dt_box, widget_init_hmm.init_n_start, widget_init_hmm.init_n_end)

Text(value='', description='Directory', placeholder='directory to be searched in', style=DescriptionStyle(desc…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

Text(value='0.02', description='Camera integration time [s]', placeholder='Define time between two acquired fr…

Text(value='1', description='Min number of states', placeholder='Define an integer', style=DescriptionStyle(de…

Text(value='4', description='Max number of states', placeholder='Define an integer', style=DescriptionStyle(de…

In [3]:
display(widget_init_hmm.run_button)
init_hmm = hiddenMarkovModeling.InitHMM()
def run_init_analysis(event):
    widget_init_hmm.create_clear_output()
    init_hmm.clear_init()
    display(widget_init_hmm.run_button)
    init_hmm.dir_path = widget_init_hmm.dir_box.value
    init_hmm.dt = float(widget_init_hmm.dt_box.value)
    init_hmm.n_start = int(widget_init_hmm.init_n_start.value)
    init_hmm.n_end = int(widget_init_hmm.init_n_end.value)
    init_hmm.run_scores()
    init_hmm.get_average_params()
widget_init_hmm.run_button.on_click(run_init_analysis)

Button(description='run', style=ButtonStyle(), tooltip='run the analysis')

### Plot cells
Visualize the jump distance mixture model per cell. Choose a number of states, define the maximum x axis range, and display the fit per cell.

In [4]:
display(widget_init_hmm.n_states_box, widget_init_hmm.x_axis_box)
def dropdown(event):
    widget_init_hmm.drop_down_cells.options = init_hmm.names
widget_init_hmm.run_button.on_click(dropdown)
display(widget_init_hmm.drop_down_cells)

Text(value='1', description='Number of states', placeholder='e.g. with best scores', style=DescriptionStyle(de…

Text(value='300', description='x axis range [nm]', placeholder='max range of x axis in nm', style=DescriptionS…

Dropdown(description='Cell:', options=(), value=None)

In [5]:
def plot_cell(event):
    widget_init_hmm.create_clear_output()
    display(widget_init_hmm.plot_button)
    try:
        init_hmm.plot_cell_results(int(widget_init_hmm.n_states_box.value), widget_init_hmm.drop_down_cells.value, int(widget_init_hmm.x_axis_box.value))
    except ValueError:
        print("Choose a number of state within the min/max range.")
display(widget_init_hmm.plot_button)
widget_init_hmm.plot_button.on_click(plot_cell)

Button(description='plot', style=ButtonStyle(), tooltip='plot results for chosen number of states')

### Save results
Define a folder name and directory to save the results and a figure format (png, svg, pdf ...). The information critera, weights and diffusion coefficients per number of states are saved. The plots per cell are saved in the SPTAnalyser_cellname/hmm subfolder.

In [6]:
# Save results
display(widget_init_hmm.box_foldername, widget_init_hmm.figure_type_box, widget_init_hmm.dir_box_save, widget_init_hmm.dir_button_save)
widget_init_hmm.dir_button_save.on_click(widget_init_hmm.open_dir_save)
widget_init_hmm.dir_box_save.observe(widget_init_hmm.change_dir_box_save)

Text(value='hidden_markov_modeling', description='Foldername', placeholder='name of folder', style=Description…

Text(value='pdf', description='Figure format', placeholder='png, pdf, svg ...', style=DescriptionStyle(descrip…

Text(value='', description='Directory', placeholder='Directory to save', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [7]:
# Save results
display(widget_init_hmm.save_button)
def save_init_analysis(event):
    widget_init_hmm.create_clear_output()
    display(widget_init_hmm.save_button)
    save_init_HMM = saveHMM.SaveInitHMM(widget_init_hmm.dir_box_save.value + "\\" + widget_init_hmm.box_foldername.value, widget_init_hmm.dir_box.value, init_hmm, widget_init_hmm.figure_type_box.value, int(widget_init_hmm.x_axis_box.value))
    save_init_HMM.save()
    print("Results are saved at", widget_init_hmm.dir_box_save.value + "\\" + widget_init_hmm.box_foldername.value)
widget_init_hmm.save_button.on_click(save_init_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Train hidden Markov model
Train a hidden Markov model with fixed number of hidden states.

Initial parameters are required for the HMM. Make sure that the number of states match the number of given initial parameters of diffusion coefficients and weights.

*Directory:* All tracked files will be analyzed within defined input directory. <br/>
*Graphviz bin:* Define path to Graphviz installation bin file (e.g. C:\Program Files (x86)\Graphviz2.38\bin). <br/>
*Camera integration time [s]:* The camera integration time of measurements in seconds. <br/>
*Number of states:* Number of hidden states. <br/>
*Min trajectory length:* Trajectories with a length [frames] below this threshold are filtered out. <br/>
*Initial diffusion coefficients:* Initial guesses about diffusion coefficients, please separate values with commas. <br/>
*Intial weights:* Initial guesses about state population, please separate values with commas. Make sure that the weights sum up to one.  <br/>
*Stability:* Initial guesses about the transition probabilities. The stability equals transitions within the same state (1->1, 2->2, ...) and are more stable then transitions between different states. All transition possibilities from a state sum to 1 (example of a three state model: 1->1 = 0.9, 1->2 = 0.05, 1->3 = 0.05). <br/>
*Train x*: Check the parameters that should be trained. Otherwise the initial parameters are fixed. <br/>
*Node size:* Choose the node size reference of the state transition diagram. The jdd fit weights are fixed diffusion coefficients from the trained HMM used in a jump distance mixture model that yields the weights. The state probabilities are directly yielded from the trained HMMs initial probabilities. The occurence are the counts in percent of labeled jumps of the observed sequences by the Viterbi algorithm.  <br/>

In [8]:
display(widget_hmm.dir_box, widget_hmm.dir_button)
widget_hmm.dir_button.on_click(widget_hmm.open_dir)
widget_hmm.dir_box.observe(widget_hmm.change_dir_box)
display(widget_hmm.dir_graphviz_box, widget_hmm.dir_graphviz_button)
widget_hmm.dir_graphviz_button.on_click(widget_hmm.open_graphviz_dir)
widget_hmm.dir_graphviz_box.observe(widget_hmm.change_dir_graphviz_box)
display(widget_hmm.dt_box, widget_hmm.n_states_box, widget_hmm.min_len_box, widget_hmm.D_box, widget_hmm.w_box, widget_hmm.stability_box, widget_hmm.train_ws_checkbox, widget_hmm.train_ds_checkbox, widget_hmm.train_tps_checkbox, widget_hmm.choose_weights_button)

Text(value='', description='Directory', placeholder='directory to be searched in', style=DescriptionStyle(desc…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

Text(value='C:\\Program Files (x86)\\Graphviz2.38\\bin', description='Graphviz bin', placeholder='define path …

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

Text(value='0.02', description='Camera integration time [s]', placeholder='Define time between two acquired fr…

...

Text(value='0.9', description='Stability', placeholder='Probability of staying in the state per timestep', sty…

Checkbox(value=True, description='Train weights', indent=False)

Checkbox(value=True, description='Train diffusion coefficients', indent=False)

Checkbox(value=True, description='Train transition probabilities', indent=False)

RadioButtons(description='Node size', layout=Layout(width='max-content'), options=('jdd fit weights', 'occurre…

In [9]:
display(widget_hmm.run_button)
hmm = hiddenMarkovModeling.HMM()
def run_analysis(event):
    widget_hmm.create_clear_output()
    hmm.clear_init()
    display(widget_hmm.run_button)
    os.environ["PATH"] += os.pathsep + widget_hmm.dir_graphviz_box.value
    hmm.dir_path = widget_hmm.dir_box.value
    hmm.dt = float(widget_hmm.dt_box.value)
    hmm.n_states = int(widget_hmm.n_states_box.value)
    hmm.min_len = int(widget_hmm.min_len_box.value)
    hmm.color_palette = color_palette_hex
    x = np.asarray([float(i) for i in widget_hmm.D_box.value.split(",")])
    y = [[] for _ in range(int(widget_hmm.n_states_box.value))]
    for i in range(len(y)):
        y[i].append(x[i])
    hmm.init_Ds = np.asarray([np.asarray(i) for i in y])
    print("The diffusion coefficents are initialized:", hmm.init_Ds, sep="\n")
    hmm.init_Ds = np.multiply(hmm.init_Ds, 1000000)
    hmm.init_ws = np.asarray([float(i) for i in widget_hmm.w_box.value.split(",")])
    if np.sum(hmm.init_ws) != 1:
        hmm.init_ws[-1] = 1-np.sum(hmm.init_ws[:-1])
        print("The weights are corrected to sum up to 1:", hmm.init_ws, sep="\n")
    else:
        print("The weights are initialized:", hmm.init_ws, sep="\n")
    hmm.init_tps = em.init_transition_matrix(n_components=int(widget_hmm.n_states_box.value), stability=float(widget_hmm.stability_box.value))
    # want to have a non symmetric transition matrix? Just change the values by indexing, e.g. hmm.init_tps[0][0] = 0.85,
    # make sure that rows sum up to one:
    # hmm.init_tps[0][0] = 0.85
    # hmm.init_tps[0][1] = 0.15
    print("The transition probability matrix is initialized:", hmm.init_tps, sep="\n")
    hmm.run([widget_hmm.train_ws_checkbox.value, widget_hmm.train_tps_checkbox.value, widget_hmm.train_ds_checkbox.value], widget_hmm.choose_weights_button.value, widget_hmm.dir_box.value)
widget_hmm.run_button.on_click(run_analysis)

Button(description='run', style=ButtonStyle(), tooltip='run the analysis')

### Check for state immobility
For each state NeNA is calculated and compared to the immobility threshold (derived from localization uncertainty during measurement). If NeNA < threshold, the state can be called immobile.

In [10]:
def check_immob(event):
    widget_hmm.create_clear_output()
    display(widget_hmm.immob_threshold_box, widget_hmm.immob_check_button)
    hmm.run_immob_check(float(widget_hmm.immob_threshold_box.value))
display(widget_hmm.immob_threshold_box, widget_hmm.immob_check_button)
widget_hmm.immob_check_button.on_click(check_immob)

Text(value='25', description='Immobile threshold [nm]', placeholder='Insert values comma separated', style=Des…

Button(description='run', style=ButtonStyle(), tooltip='check if apparent diffusion states by HMM are immobile…

### Correct diffusion coefficients for static and dynamic errors
The apparent diffusion coefficients from HMM can be corrected for static and dynamic errors. Please define a static error (NeNA, Mortensen ...).

In [11]:
def d_corr(event):
    widget_hmm.create_clear_output()
    display(widget_hmm.epsilon_box, widget_hmm.create_d_corr_button)
    hmm.run_correction(float(widget_hmm.epsilon_box.value))
display(widget_hmm.epsilon_box, widget_hmm.create_d_corr_button)
widget_hmm.create_d_corr_button.on_click(d_corr)

Text(value='20', description='Static error [nm]', placeholder='Insert values comma separated', style=Descripti…

Button(description='run', style=ButtonStyle(), tooltip='correct diffusion coefficients by dynamic and static e…

### Plot cells
A trained jump-distance mixed model with only the weight parameters optimized and the expected diffusion coefficients fixed by the HMM. Plot the trained model with the jumpdistance distribution of the cell.

In [12]:
display(widget_hmm.x_axis_box)
def dropdown_hmm(event):
    widget_hmm.drop_down_cells.options = hmm.names
widget_hmm.run_button.on_click(dropdown_hmm)
display(widget_hmm.drop_down_cells)

Text(value='300', description='x axis range [nm]', placeholder='max range of x axis in nm', style=DescriptionS…

Dropdown(description='Cell:', options=(), value=None)

In [13]:
def plot_hmm_cell(event):
    widget_hmm.create_clear_output()
    display(widget_hmm.plot_button)
    hmm.plot_cell_fit(widget_hmm.drop_down_cells.value, int(widget_hmm.x_axis_box.value))
display(widget_hmm.plot_button)
widget_hmm.plot_button.on_click(plot_hmm_cell)

Button(description='plot', style=ButtonStyle(), tooltip='plot results for chosen number of states')

### Save results
Define a folder name and directory to save the results and a figure format (png, svg, pdf ...). The information critera, plots, and HMM parameters are saved in the defined folder. The plots per cell and the state labels per jump distance are saved in the SPTAnalyser_cellname/hmm subfolder.

In [14]:
# Save results
display(widget_hmm.box_foldername, widget_hmm.figure_type_box, widget_hmm.dir_box_save, widget_hmm.dir_button_save)
widget_hmm.dir_button_save.on_click(widget_hmm.open_dir_save)
widget_hmm.dir_box_save.observe(widget_hmm.change_dir_box_save)

Text(value='hidden_markov_modeling', description='Foldername', placeholder='name of folder', style=Description…

Text(value='pdf', description='Figure format', placeholder='png, pdf, svg ...', style=DescriptionStyle(descrip…

Text(value='', description='Directory', placeholder='Directory to save', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [15]:
# Save results
display(widget_hmm.save_button)
def save_init_analysis(event):
    widget_hmm.create_clear_output()
    display(widget_hmm.save_button)
    save_hmm = saveHMM.SaveHMM(widget_hmm.dir_box_save.value + "\\" + widget_hmm.box_foldername.value, widget_hmm.dir_box.value, hmm, widget_hmm.figure_type_box.value, int(widget_hmm.x_axis_box.value))
    save_hmm.save(widget_hmm.choose_weights_button.value)
    print("Results are saved at", widget_hmm.dir_box_save.value + "\\" + widget_hmm.box_foldername.value)
widget_hmm.save_button.on_click(save_init_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')